# Numba

Numba is a just-in-time compiler (__JIT__) for Python that aims at speeding up array-orianted and math-heavy programs. In contrast to Cython, which follows an ahead-of-time approach and depends on additional type information, Numba tries to gather type information via __type inference__ during the first execution of a function and then uses this information to generate optimized machine code for subsequent executions. In the backend, the [__LLVM__](https://llvm.org/) compiler infrastructure is working hard to generate machine code targeted at the currently used CPU.

## Motivation

In [1]:
import numba
from numba import float32, float64, int8, int32, void, prange
import numpy as np
import random
import timeit
from math import sqrt

In [2]:
def monte_carlo_pi_python(n):
    hits = 0

    for i in range(n):
        x = random.random()
        y = random.random()

        if (sqrt(x**2 + y**2) <= 1.0):
            hits += 1

    return 4*hits/n

In [3]:
%timeit -r 10 -n 30 monte_carlo_pi_python(10_000)

3.66 ms ± 61 µs per loop (mean ± std. dev. of 10 runs, 30 loops each)


In [4]:
monte_carlo_pi_python(2000_000)

3.141298

In [5]:
@numba.jit
def monte_carlo_pi_numba(n):
    hits = 0

    for i in range(n):
        x = random.random()
        y = random.random()

        if ((x**2 + y**2) < 1.0):
            hits += 1

    return 4*hits/n

In [6]:
%timeit -r 10 -n 30 monte_carlo_pi_numba(10_000)

The slowest run took 147.17 times longer than the fastest. This could mean that an intermediate result is being cached.
1.67 ms ± 4.69 ms per loop (mean ± std. dev. of 10 runs, 30 loops each)


## Type Inference

The primary mechanism by which Numba is invoked is the decorator `@jit`. By simply putting this decorator above a function definition, the compiler will automatically try to optimize it with the first program execution. Naturally, if Numba can infer enough enough information and succeeds with the optimization, this very first execution is significantly slower than the subsequent ones. Depending on how the optimized function is called, Numba might generate different versions of each function. Furthermore, all functions called by Numba-optimized functions need to be Numba-optimized themselves in order to gain the full potential.

In [7]:
@numba.jit()
def multlist(xs, y, i):
    xs[i] = xs[i]*y

    
@numba.jit()
def domath(xs, a, b, c):
    for i in range(xs.shape[0]):
        multlist(xs, a+b, i)
        multlist(xs, c, i)

In [8]:
a = 1
b = 2
c = 3
xs = np.ones(100_000, dtype=np.int32)

%timeit -r 30 -n 30 domath(xs, a, b, c)

The slowest run took 453.29 times longer than the fastest. This could mean that an intermediate result is being cached.
121 µs ± 613 µs per loop (mean ± std. dev. of 30 runs, 30 loops each)


In [9]:
domath.inspect_types()

domath (array(int32, 1d, C), int64, int64, int64)
--------------------------------------------------------------------------------
# File: /tmp/ipykernel_318802/3499217220.py
# --- LINE 6 --- 

@numba.jit()

# --- LINE 7 --- 

def domath(xs, a, b, c):

    # --- LINE 8 --- 
    # label 0
    #   xs = arg(0, name=xs)  :: array(int32, 1d, C)
    #   a = arg(1, name=a)  :: int64
    #   b = arg(2, name=b)  :: int64
    #   c = arg(3, name=c)  :: int64
    #   $2load_global.0 = global(range: <class 'range'>)  :: Function(<class 'range'>)
    #   $6load_attr.2 = getattr(value=xs, attr=shape)  :: UniTuple(int64 x 1)
    #   $const8.3 = const(int, 0)  :: Literal[int](0)
    #   $10binary_subscr.4 = static_getitem(value=$6load_attr.2, index=0, index_var=$const8.3, fn=<built-in function getitem>)  :: int64
    #   del $const8.3
    #   del $6load_attr.2
    #   $12call_function.5 = call $2load_global.0($10binary_subscr.4, func=$2load_global.0, args=[Var($10binary_subscr.4, 3499217220.py:8)], kw

In [10]:
multlist.inspect_types()

multlist (array(int32, 1d, C), int64, int64)
--------------------------------------------------------------------------------
# File: /tmp/ipykernel_318802/3499217220.py
# --- LINE 1 --- 

@numba.jit()

# --- LINE 2 --- 

def multlist(xs, y, i):

    # --- LINE 3 --- 
    # label 0
    #   xs = arg(0, name=xs)  :: array(int32, 1d, C)
    #   y = arg(1, name=y)  :: int64
    #   i = arg(2, name=i)  :: int64
    #   $6binary_subscr.2 = getitem(value=xs, index=i, fn=<built-in function getitem>)  :: int32
    #   $10binary_multiply.4 = $6binary_subscr.2 * y  :: int64
    #   del y
    #   del $6binary_subscr.2
    #   xs[i] = $10binary_multiply.4  :: (array(int32, 1d, C), int64, int64) -> none
    #   del xs
    #   del i
    #   del $10binary_multiply.4
    #   $const18.7 = const(NoneType, None)  :: none
    #   $20return_value.8 = cast(value=$const18.7)  :: none
    #   del $const18.7
    #   return $20return_value.8

    xs[i] = xs[i]*y




## Lazy versus Eager Compilation

Even though Numba is mostly able to infer enough type information by itself, sometimes it is preferable to add type information to the source code. Without any given type information, Numba necessarily deferrs compilation until the first function execution, because only then are the types of the function parameters , i.e. the function signature, known. This is called __lazy compilation__. On the other hand, with __eager compilation__ Numba compiles as soon as the function is imported and uses the given function signature and compiles the function only for the given signature, which might be useful if tight control over the chosen types is desired, e.g. floats are preferred ovder doubles.

In [11]:
@numba.jit(void(float64, float64[:]))
def multy(y, xs):
    for i in range(len(xs)):
        xs[i] = y*xs[i]

In [12]:
@numba.jit(void(int32, int32[:]))
def multy(y, xs):
    for i in range(len(xs)):
        xs[i] = y*xs[i]

In [13]:
multy.inspect_types()

multy (int32, array(int32, 1d, A))
--------------------------------------------------------------------------------
# File: /tmp/ipykernel_318802/2525040080.py
# --- LINE 1 --- 

@numba.jit(void(int32, int32[:]))

# --- LINE 2 --- 

def multy(y, xs):

    # --- LINE 3 --- 
    # label 0
    #   y = arg(0, name=y)  :: int32
    #   xs = arg(1, name=xs)  :: array(int32, 1d, A)
    #   $2load_global.0 = global(range: <class 'range'>)  :: Function(<class 'range'>)
    #   $4load_global.1 = global(len: <built-in function len>)  :: Function(<built-in function len>)
    #   $8call_function.3 = call $4load_global.1(xs, func=$4load_global.1, args=[Var(xs, 2525040080.py:3)], kws=(), vararg=None, target=None)  :: (array(int32, 1d, A),) -> int64
    #   del $4load_global.1
    #   $10call_function.4 = call $2load_global.0($8call_function.3, func=$2load_global.0, args=[Var($8call_function.3, 2525040080.py:3)], kws=(), vararg=None, target=None)  :: (int64,) -> range_state_int64
    #   del $8call_fu

In [ ]:
xs = np.ones(100_000, np.float64)

%timeit -r 30 -n 100 multy(np.float64('2'), xs)

In [15]:
xs = np.ones(100_000, np.int32)

%timeit -r 30 -n 100 multy(np.int32('2'), xs)

43.3 µs ± 1.54 µs per loop (mean ± std. dev. of 30 runs, 100 loops each)


In [ ]:
xs = np.ones(100_000, np.int8)

%timeit -r 30 -n 100 multy(np.int8('2'), xs)

## Compilation Modes

Numba comes with two compilation modes: `object` and `nopython` mode. In `nopython` mode the behaviour is such that decorated function will run entirely without involvement of the Python interpreter but it requires that the types of all variables in the function can be unambiguously inferred. The best-performing code will generally be produced in this mode. Compilation in `object` mode means that the Numba compiler generates code under the assumption that all values are Python objects and involves the Python interpreter for operations on those objects. While this code in general runs faster than native, interpreted Python code, it will be less performant than `nopython` code.

In [17]:
@numba.jit("void(int32[:])", nopython=True)
def bubblesort(xs):
    n = xs.shape[0]
    for i in range(n-1):
        for j in range(n-i-1):
            if xs[j + 1] < xs[j]:
                xs[j], xs[j + 1] = xs[j + 1], xs[j]

In [18]:
xs = np.random.randint(0, high=1000, size=100, dtype=np.int32)

%timeit -r 30 -n 100 bubblesort(xs)

11.3 µs ± 707 ns per loop (mean ± std. dev. of 30 runs, 100 loops each)


In [19]:
from decimal import Decimal


@numba.jit("void(int32[:])", nopython=False)
def bubblesort(xs):
    n = xs.shape[0]
    d = Decimal(0)
    for i in range(n-1):
        for j in range(n-i-1):
            if xs[j + 1] < xs[j]:
                xs[j], xs[j + 1] = xs[j + 1], xs[j]

/tmp/ipykernel_318802/3977328882.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "bubblesort" failed type inference due to: Untyped global name 'Decimal': Cannot determine Numba type of <class 'type'>

File "../../../../tmp/ipykernel_318802/3977328882.py", line 7:
<source missing, REPL/exec in use?>

  @numba.jit("void(int32[:])", nopython=False)
/tmp/ipykernel_318802/3977328882.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "bubblesort" failed type inference due to: Untyped global name 'Decimal': Cannot determine Numba type of <class 'type'>

File "../../../../tmp/ipykernel_318802/3977328882.py", line 7:
<source missing, REPL/exec in use?>

  @numba.jit("void(int32[:])", nopython=False)
/opt/sw/vsc4/VSC/x86_64/generic/jupyterhub-envs/conda/envs/jupyterhub-dask/lib/python3.10/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "bubblesort" was c

## Parallelization

Numba also tries its best to fullfill the dream of every programmer, namely automatic parallelization. By setting the parameter `parallel=True` in the `jit` decorator, Numba performs program transformations that introduce parallelism to a noticable degree. The documentation lists all [supported operations with parallel semantics](https://numba.readthedocs.io/en/stable/user/parallel.html#supported-operations) for which Numba attempts to parallelize.
In order to hint to Numba that a loop can be parallelized, the `prange` loop is used instead of Python's native `range` loop and it is the programmer's responsibility to prevent dependencies across iterations of parallel loops.

In [20]:
@numba.jit("void(float64[:], float64[:], float64[:], float64[:])", nopython=True, parallel=True)
def diffquots(xs, fdxs, cdxs, bdxs):
    fdxs[0] = 0
    cdxs[0] = 0
    bdxs[0] = 0
    for i in prange(1, xs.shape[0]-1):
        fdxs[i] = np.exp(-0.5*(xs[i-1] - xs[i]))
        cdxs[i] = np.exp(-0.5*((xs[i+1] + xs[i-1])/2.0))
        bdxs[i] = np.exp(-0.5*(xs[i] - xs[i-1]))

In [21]:
n = 10**4
xs = np.random.rand(n)
fdxs = np.empty_like(xs)
cdxs = np.empty_like(xs)
bdxs = np.empty_like(xs)

diffquots(xs, fdxs, cdxs, bdxs)

In [22]:
%timeit -r 30 -n 10 diffquots(xs, fdxs, cdxs, bdxs)

The slowest run took 146.63 times longer than the fastest. This could mean that an intermediate result is being cached.
538 µs ± 2.4 ms per loop (mean ± std. dev. of 30 runs, 10 loops each)


In [23]:
@numba.jit(parallel=True)
def addsmt(xs):
    for i in prange(1, xs.shape[0]):
        xs[i] += xs[i-1]

In [24]:
xs = np.ones(100)
addsmt(xs)
print(xs)

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28.  2.  3.  4.  5.  6.  7.  8.  9.
 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25.  2.  3.
  4.  5.  6.  7.  8.  9.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.
 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31.
 32. 33.  2.  3.  4.  5.  6.  7.  8.  9.]


## Numba & NumPy's ufuncs

Two types of universal functions are supported by Numba: universal functions that work on scalars and generalized universal functions that work on multi-dimensional arrays as well.
For universal functions, the decorator `@vectorize` is required and it supports eager compilation with several signatures as well as lazy compilation. 


The decorator `@guvectorize` for generalized universal functions allows to write universal functions that work on any number of elements in an inpute array and return arrays with different dimensions as the input arrays. These functions do not return any values, instead their output must be written to a given array. Just as the simpler `@vectorize` functions, `@guvectorize` functions can be compiled eagerly or lazyl.

In [25]:
@numba.vectorize([float64(float64, float64)])
def f(x, y):
    return x + y + 7

In [26]:
ls = np.ones(100_000)
ks = np.ones(100_000)

%timeit -r 30 -n 10 f(ls, ks)

147 µs ± 12.9 µs per loop (mean ± std. dev. of 30 runs, 10 loops each)


In [27]:
@numba.vectorize([int8(int8, int8),
                  int32(int32, int32),
                  float32(float32, float32),
                  float64(float64, float64)])
def g(x, y):
    return x * y * 5

In [28]:
ls = np.ones(100_000, dtype=np.int64)
ks = np.ones(100_000, dtype=np.float64)

%timeit -r 30 -n 10 g(ls, ks)

151 µs ± 4.99 µs per loop (mean ± std. dev. of 30 runs, 10 loops each)


In [29]:
@numba.guvectorize([(int32[:], int32, int32[:])], '(n),()->(n)')
def h(x, y, res):
    for i in range(x.shape[0]):
        res[i] = x[i] + y

In [30]:
x = np.ones(100_000, dtype=np.int32)
y = 1
res = np.empty_like(x)

%timeit -r 30 -n 10 h(x, y, res)

47.4 µs ± 7.43 µs per loop (mean ± std. dev. of 30 runs, 10 loops each)


## Danger Zone

Despite Numba's extensive support for Python and Numpy features, two unapparent 

* Python's containers are not thread-safe and therefore must be handled with care in parallel programs.
* The loop variable introduced into the program by `prange` loops must not be used as an index, since the compiler does not make any guarantees about which indices are assigned to which threads or the execution order of the iterations.

In [31]:
@numba.jit(parallel=True)
def parappend():
    ls = []
    for i in prange(10**6):
        ls.append(i)
    return ls

In [32]:
@numba.jit(parallel=True)
def invalid():
    n = get_num_threads()
    z = [0 for _ in range(n)]
    for i in prange(100):
        z[i % n] += i
    return z